In [18]:
# ==============================================================================
# بخش ۱: نصب و ورود کتابخانه‌ها
# ==============================================================================
print("Installing required libraries for interactive mapping...")
!pip install earthengine-api pandas numpy folium ipyleaflet ipywidgets -q
import ee
import pandas as pd
import numpy as np
import folium
from ipyleaflet import Map, Marker, basemaps, DivIcon
from ipywidgets import Button, Output, VBox, HBox
from IPython.display import display

print("Installation and imports complete.")

# ==============================================================================
# بخش ۲: احراز هویت
# ==============================================================================
YOUR_PROJECT_ID = 'ee-maziarasmani'
try:
    ee.Initialize(project=YOUR_PROJECT_ID)
    print(f"Successfully initialized Earth Engine for project: {YOUR_PROJECT_ID}")
except Exception as e:
    print("Authenticating and initializing Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project=YOUR_PROJECT_ID)
    print(f"Authentication successful. Initialized for project: {YOUR_PROJECT_ID}")

# ==============================================================================
# بخش ۳: توابع تحلیل (نسخه ۹ معیاره)
# ==============================================================================
def get_gee_data_9_criteria(locations):
    """Extracts 9 globally available criteria for any list of locations."""
    print("Preparing server-side computation (9-Criteria Version)...")
    user_points = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point(loc['lon'], loc['lat']), {'name': loc['name'], 'lat': loc['lat'], 'lon': loc['lon']})
        for loc in locations
    ])

    # Define necessary global datasets
    srtm = ee.Image('USGS/SRTMGL1_003')
    gldas_collection = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filterDate('2023-01-01', '2023-12-31').select(
        ['Tair_f_inst', 'SWdown_f_tavg', 'Wind_f_inst', 'Qair_f_inst', 'Psurf_f_inst', 'Rainf_f_tavg']
    )
    modis_aod = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES').filterDate('2023-01-01', '2023-12-31').select('Optical_Depth_055')

    def extract_values(point_feature):
        point_geometry = point_feature.geometry()

        slope_dict = ee.Terrain.slope(srtm).reduceRegion(reducer=ee.Reducer.mean(), geometry=point_geometry, scale=90)
        elevation_dict = srtm.select('elevation').reduceRegion(reducer=ee.Reducer.mean(), geometry=point_geometry, scale=90)
        gldas_dict = gldas_collection.mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=point_geometry, scale=27830)
        aod_dict = modis_aod.mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=point_geometry, scale=1000)

        # Safely get values using server-side conditional logic
        slope = ee.Algorithms.If(slope_dict.contains('slope'), slope_dict.get('slope'), None)
        elevation = ee.Algorithms.If(elevation_dict.contains('elevation'), elevation_dict.get('elevation'), None)
        temp_k = ee.Algorithms.If(gldas_dict.contains('Tair_f_inst'), gldas_dict.get('Tair_f_inst'), None)
        radiation_w_m2 = ee.Algorithms.If(gldas_dict.contains('SWdown_f_tavg'), gldas_dict.get('SWdown_f_tavg'), None)
        wind_speed = ee.Algorithms.If(gldas_dict.contains('Wind_f_inst'), gldas_dict.get('Wind_f_inst'), None)
        humidity_specific = ee.Algorithms.If(gldas_dict.contains('Qair_f_inst'), gldas_dict.get('Qair_f_inst'), None)
        pressure_pa = ee.Algorithms.If(gldas_dict.contains('Psurf_f_inst'), gldas_dict.get('Psurf_f_inst'), None)
        precipitation_flux = ee.Algorithms.If(gldas_dict.contains('Rainf_f_tavg'), gldas_dict.get('Rainf_f_tavg'), None)
        aod = ee.Algorithms.If(aod_dict.contains('Optical_Depth_055'), aod_dict.get('Optical_Depth_055'), None)

        return point_feature.set({
            'Land Slope': slope, 'Elevation': elevation, 'Ambient Temperature_K': temp_k, 'Solar Irradiance_W_m2': radiation_w_m2,
            'Wind Speed': wind_speed, 'Specific Humidity': humidity_specific, 'Air Pressure_Pa': pressure_pa,
            'Precipitation_Flux': precipitation_flux, 'Dust Level (AOD)': aod
        })

    print("Executing computation on Google's servers...")
    results_collection = user_points.map(extract_values).getInfo()
    print("Data extraction complete.")

    criteria_data = []
    for feature in results_collection['features']:
        props = feature['properties']

        temp_c = (props.get('Ambient Temperature_K') - 273.15) if props.get('Ambient Temperature_K') is not None else None
        radiation_w = props.get('Solar Irradiance_W_m2')
        radiation_kwh_day = (radiation_w * 24 / 1000) if radiation_w is not None else None
        aod_scaled = props.get('Dust Level (AOD)') * 1000 if props.get('Dust Level (AOD)') is not None else None
        pressure_pa = props.get('Air Pressure_Pa')
        precip_flux = props.get('Precipitation_Flux')
        precip_annual_mm = (precip_flux * 3600 * 24 * 365) if precip_flux is not None else None

        relative_humidity = None
        q = props.get('Specific Humidity')
        if q is not None and temp_c is not None and pressure_pa is not None:
            es = 610.94 * np.exp((17.625 * temp_c) / (243.04 + temp_c))
            e_vapor_pressure = (q * pressure_pa) / (0.622 + (0.378 * q))
            rh = 100 * (e_vapor_pressure / es)
            relative_humidity = min(rh, 100.0)

        criteria_data.append({
            'Location': props['name'], 'Latitude': props['lat'], 'Longitude': props['lon'],
            'Solar Irradiance (kWh/m²/day)': radiation_kwh_day,
            'Ambient Temperature (°C)': temp_c,
            'Land Slope (°)': props.get('Land Slope'),
            'Dust Level (AOD)': aod_scaled,
            'Relative Humidity (%)': relative_humidity,
            'Wind Speed (m/s)': props.get('Wind Speed'),
            'Elevation (m)': props.get('Elevation'),
            'Air Pressure (hPa)': pressure_pa / 100 if pressure_pa is not None else None,
            'Precipitation (mm/year)': precip_annual_mm
        })

    return pd.DataFrame(criteria_data).set_index('Location')

def calculate_shannon_weights(dataframe):
    dataframe = dataframe.dropna()
    if len(dataframe) < 2: raise ValueError("Not enough valid data for analysis.")
    p_matrix = dataframe / dataframe.sum(axis=0); num_alternatives = len(dataframe)
    k = 1 / np.log(num_alternatives); entropy_values = -k * (p_matrix * np.log(p_matrix.replace(0, 1e-12))).sum(axis=0)
    divergence = 1 - entropy_values; weights = divergence / divergence.sum()
    return weights, pd.DataFrame({'Entropy': entropy_values, 'Divergence': divergence, 'Weight': weights}).sort_values(by='Weight', ascending=False)

def run_topsis_ranking(dataframe, weights, criteria_types):
    dataframe = dataframe.dropna()
    if len(dataframe) < 2: raise ValueError("Not enough valid data for analysis.")
    r_matrix = dataframe / np.sqrt((dataframe**2).sum(axis=0)); v_matrix = r_matrix * weights
    ideal_positive = pd.Series(index=v_matrix.columns, dtype=float); ideal_negative = pd.Series(index=v_matrix.columns, dtype=float)
    for i, col in enumerate(v_matrix.columns):
        if criteria_types[i] == 'benefit': ideal_positive[col] = v_matrix[col].max(); ideal_negative[col] = v_matrix[col].min()
        else: ideal_positive[col] = v_matrix[col].min(); ideal_negative[col] = v_matrix[col].max()
    s_positive = np.sqrt(((v_matrix - ideal_positive)**2).sum(axis=1)); s_negative = np.sqrt(((v_matrix - ideal_negative)**2).sum(axis=1))
    closeness = s_negative / (s_positive + s_negative)
    results_df = pd.DataFrame({'S+': s_positive, 'S-': s_negative, 'Ci': closeness})
    results_df['Rank'] = results_df['Ci'].rank(ascending=False, method='first').astype(int)
    return results_df.sort_values(by='Rank')

print("Analysis functions are defined.")

# ==============================================================================
# بخش ۴: ساخت ابزار تعاملی
# ==============================================================================
clicked_points = []
point_counter = 0
markers_on_map = []

center = [32.4279, 53.6880]
zoom = 5
interactive_map = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldImagery)
output_widget = Output()
analyze_button = Button(description="شروع تحلیل (Analyze)", button_style='success', icon='cogs')
reset_button = Button(description="ریست (Reset)", button_style='warning', icon='refresh')

def on_map_click(type, coordinates, **kwargs):
    global point_counter
    if type == 'click':
        point_counter += 1
        clicked_points.append({'name': f'Site {point_counter}', 'lat': coordinates[0], 'lon': coordinates[1]})
        icon = DivIcon(
            html=f'<div style="font-size: 12px; font-weight: bold; color: white; background-color: #007bff; border-radius: 50%; width: 24px; height: 24px; text-align: center; line-height: 24px; border: 2px solid white;">{point_counter}</div>',
            icon_size=(24, 24), icon_anchor=(12, 12)
        )
        marker = Marker(location=coordinates, icon=icon, draggable=False)
        interactive_map.add_layer(marker)
        markers_on_map.append(marker)
        analyze_button.description = f"تحلیل {point_counter} نقطه (Analyze)"

def on_button_click(b):
    with output_widget:
        output_widget.clear_output(wait=True)
        if len(clicked_points) < 2:
            print("❌ ERROR: Please select at least 2 points on the map before analyzing.")
            return

        print("Starting analysis for the selected points...")
        try:
            decision_matrix_raw = get_gee_data_9_criteria(clicked_points)
            print("\n--- Raw Data Extracted for All Points (Before Cleaning) ---")
            display(decision_matrix_raw)

            decision_matrix_clean = decision_matrix_raw.dropna()

            if len(decision_matrix_clean) < 2:
                print("\n❌ ERROR: Fewer than 2 valid locations could be processed after data cleaning.")
                return

            decision_matrix = decision_matrix_clean.drop(columns=['Latitude', 'Longitude'])
            CRITERIA_TYPES = [
                'benefit', 'cost', 'cost', 'cost', 'cost', 'benefit',
                'cost', 'benefit', 'cost'
            ]

            weights, weights_df = calculate_shannon_weights(decision_matrix)
            final_ranking = run_topsis_ranking(decision_matrix, weights, CRITERIA_TYPES)

            print("\n--- Criteria Weighting Results ---")
            display(weights_df)
            print("\n--- Final Ranking Results ---")
            display(final_ranking)

            print("\nGenerating final results map...")
            plot_data = final_ranking.merge(decision_matrix_clean[['Latitude', 'Longitude']], left_index=True, right_index=True)
            map_center = [plot_data['Latitude'].mean(), plot_data['Longitude'].mean()]
            result_map = folium.Map(location=map_center, zoom_start=4)
            bounds = [[plot_data['Latitude'].min(), plot_data['Longitude'].min()], [plot_data['Latitude'].max(), plot_data['Longitude'].max()]]
            result_map.fit_bounds(bounds, padding=(15,15))

            for idx, row in plot_data.iterrows():
                rank = row['Rank']
                score = row['Ci']
                if rank == 1: icon = folium.Icon(color='green', icon='star')
                else: icon = folium.Icon(color='blue', icon='info-sign')
                popup_html = f"<b>{row.name}</b><br>Rank: {rank}<br>Score (Ci): {score:.4f}"
                folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup_html, icon=icon).add_to(result_map)

            display(result_map)

        except ee.EEException as e:
            print(f"\n❌ A GEE server error occurred. This is often a temporary issue on Google's side.")
            print("Please try running again in a few minutes.")
            print(f"Error details: {e}")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}")

def on_reset_click(b):
    """Handles clicks on the reset button."""
    global clicked_points, point_counter, markers_on_map
    with output_widget:
        for marker in markers_on_map:
            interactive_map.remove_layer(marker)

        clicked_points = []
        point_counter = 0
        markers_on_map = []

        analyze_button.description = "شروع تحلیل (Analyze)"
        output_widget.clear_output()
        print("Map and selections have been reset. You can now select new points.")

interactive_map.on_interaction(on_map_click)
analyze_button.on_click(on_button_click)
reset_button.on_click(on_reset_click)

print("Interactive Tool Ready.")
print("1. Click on the map to select your desired locations.")
print("2. When you are done, click the 'Analyze' button.")
print("3. To start over, click the 'Reset' button.")

buttons = HBox([analyze_button, reset_button])
display(VBox([interactive_map, buttons, output_widget]))


Installing required libraries for interactive mapping...
Installation and imports complete.
Successfully initialized Earth Engine for project: ee-maziarasmani
Analysis functions are defined.
Interactive Tool Ready.
1. Click on the map to select your desired locations.
2. When you are done, click the 'Analyze' button.
3. To start over, click the 'Reset' button.
